# Working with sqlite3

In [1]:
import sqlite3

In [2]:
# create a connection to already existing db file or create a new file
conn = sqlite3.connect('employee.db')

In [3]:
# create cursor allowing us to execute sql commands
c = conn.cursor()

Now, have a look at available datatypes for sqlite3 [here](https://www.sqlite.org/datatype3.html).

In [4]:
# create an employee table
c.execute("""
CREATE TABLE employees (
    first text, 
    last text, 
    pay integer
    )
""")

# after executing the cursor, commit the CONNECTION and close it
conn.commit()
conn.close()

In [5]:
# let's start again

# get connection
conn = sqlite3.connect('employee.db')

# get cursor
c = conn.cursor()

# add data into db
c.execute("INSERT INTO employees VALUES ('Udit', 'Manav', 50000)")

# save changes
# conn.commit()

In [6]:
# let's add some more data
c.execute("INSERT INTO employees VALUES ('A', 'B', 540000)")
c.execute("INSERT INTO employees VALUES ('C', 'D', 5000)")
c.execute("INSERT INTO employees VALUES ('E', 'F', 10000)")
c.execute("INSERT INTO employees VALUES ('G', 'H', 5900)")

In [7]:
# execute a select query
c.execute("SELECT * from employees WHERE last='Manav'")
print(c.fetchone())

('Udit', 'Manav', 50000)


In [8]:
# another select query
c.execute("SELECT * from employees WHERE pay>5500")
print(c.fetchone())
print(c.fetchmany(3))
print(c.fetchall())

('Udit', 'Manav', 50000)
[('A', 'B', 540000), ('E', 'F', 10000), ('G', 'H', 5900)]
[]


What happened here is that iterator gets exhausted by the time it reached fetchall().

In [9]:
# let's try that again
c.execute("SELECT * from employees WHERE pay>5500")
print(c.fetchmany(3))

c.execute("SELECT * from employees WHERE pay>5500")
print(c.fetchall())

[('Udit', 'Manav', 50000), ('A', 'B', 540000), ('E', 'F', 10000)]
[('Udit', 'Manav', 50000), ('A', 'B', 540000), ('E', 'F', 10000), ('G', 'H', 5900)]


# Working with Classes and sqlite3

In [10]:
# create a sample Employee Class
class Employee:
    """A sample Employee class"""

    def __init__(self, first, last, pay):
        self.first = first
        self.last = last
        self.pay = pay

    @property
    def email(self):
        return '{}.{}@email.com'.format(self.first, self.last)

    @property
    def fullname(self):
        return '{} {}'.format(self.first, self.last)

    def __repr__(self):
        return "Employee('{}', '{}', {})".format(self.first, self.last, self.pay)

In [11]:
# create instances of employees
emp_1 = Employee('John', 'Doe', 80000)
emp_2 = Employee('Jane', 'Doe', 90000)

Now, for committing the values to the database, DO NOT USE FORMAT STRINGS, as they'll be prone to SQL injection attacks. Let's have look at the "safe" way to insert into DB.

In [12]:
# one way
c.execute("INSERT INTO employees VALUES (?, ?, ?)", (emp_1.first, emp_1.last, emp_1.pay))

# another way
c.execute("INSERT INTO employees VALUES (:first, :last, :pay)", {'first':emp_2.first, 'last':emp_2.last, 'pay':emp_2.pay})

In [13]:
# let's check the db content

# the second param must be tuple, even if there is only one value
c.execute("SELECT * from employees WHERE last=?", ('Doe',))  
print(c.fetchall())

c.execute("SELECT * from employees WHERE last=:last", {'last':'Manav'})  
print(c.fetchall())

[('John', 'Doe', 80000), ('Jane', 'Doe', 90000)]
[('Udit', 'Manav', 50000)]


# Tying up everything

Creating some functions that'll 
* insert emp in db
* get emp by name
* update pay
* remove emp

In [14]:
def insert_emp(emp):
    with conn:  # this will eliminate the need for commit 
        c.execute("INSERT INTO employees VALUES (:first, :last, :pay)", 
                  {'first':emp.first, 'last':emp.last, 'pay':emp.pay})


def get_emps_by_name(lastname):
    # select stmnt doesn't need to commit
    c.execute("SELECT * from employees WHERE last=:last", {'last':lastname})  
    return c.fetchall()


def update_pay(emp, pay):
    with conn:
        c.execute("""UPDATE employees SET pay = :pay
                    WHERE first = :first AND last = :last""",
                  {'first': emp.first, 'last': emp.last, 'pay': pay})


def remove_emp(emp):
    with conn:
        c.execute("DELETE from employees WHERE first = :first AND last = :last",
                  {'first': emp.first, 'last': emp.last})

In [15]:
# Let's try these out

emp_3 = Employee('X', 'Y', 999999)

# insert emp into db
insert_emp(emp_3)

# get emps by lastname
print(get_emps_by_name('Doe'))

[('John', 'Doe', 80000), ('Jane', 'Doe', 90000)]


In [16]:
# update pay of emp_2
update_pay(emp_2, 99000)

# remove emp_1
remove_emp(emp_1)

# print complete database
c.execute('SELECT * from employees')
c.fetchall()

[('Udit', 'Manav', 50000),
 ('A', 'B', 540000),
 ('C', 'D', 5000),
 ('E', 'F', 10000),
 ('G', 'H', 5900),
 ('Jane', 'Doe', 99000),
 ('X', 'Y', 999999)]

In [17]:
# Don't forget to close the connection
conn.close()

TIP: If you'd like to commit database in-memory i.e. RAM use
```Python
conn = sqlite3.connect(':memory:')
```

Also, have a look at [DB Browser for SQLite](https://sqlitebrowser.org/).